In [1]:
# import os
# import json
# import pandas as pd
# import pyarrow.parquet as pq
# import pyarrow as pa
# from tqdm import tqdm
# from pathlib import Path

# # Directory containing the JSON files
# DATA_DIR = "data"
# OUTPUT_DIR = "parquet_database"

# # Progress tracking file
# PROGRESS_FILE = "progress_log.json"
# ERROR_LOG_FILE = "error_log.json"

# # Partition column (e.g., receivedate if available)
# PARTITION_COLUMN = "receivedate"

# def load_progress():
#     """Load progress from log file."""
#     if os.path.exists(PROGRESS_FILE):
#         with open(PROGRESS_FILE, "r") as f:
#             return json.load(f)
#     return {"processed_files": [], "processed_size": 0}

# def save_progress(progress):
#     """Save progress to log file."""
#     with open(PROGRESS_FILE, "w") as f:
#         json.dump(progress, f)

# def log_error(file_path, error_message):
#     """Log errors to the error log file."""
#     error_log = []
#     if os.path.exists(ERROR_LOG_FILE):
#         with open(ERROR_LOG_FILE, "r") as f:
#             error_log = json.load(f)
#     error_log.append({"file": file_path, "error": error_message})
#     with open(ERROR_LOG_FILE, "w") as f:
#         json.dump(error_log, f, indent=4)

# def get_total_size_and_files(data_dir):
#     """Get total size and list of all JSON files."""
#     total_size = 0
#     all_files = []
#     for root, _, files in os.walk(data_dir):
#         for file in files:
#             if file.endswith(".json"):
#                 full_path = os.path.join(root, file)
#                 total_size += os.path.getsize(full_path)
#                 all_files.append(full_path)
#     return total_size, all_files

# def process_file_to_dataframe(file_path):
#     """Process a single JSON file into a Pandas DataFrame."""
#     with open(file_path, "r") as f:
#         data = json.load(f)
        
#         # Flatten JSON into a Pandas DataFrame
#         if isinstance(data, list):
#             df = pd.json_normalize(data)
#         elif isinstance(data, dict) and "results" in data:
#             df = pd.json_normalize(data["results"])
#         else:
#             df = pd.json_normalize([data])  # Single record
#         return df

# def save_to_parquet(df, output_dir, partition_column=None):
#     """Save DataFrame to Parquet with optional partitioning."""
#     table = pa.Table.from_pandas(df)
#     pq.write_to_dataset(
#         table,
#         root_path=output_dir,
#         partition_cols=[partition_column] if partition_column else None,
#         existing_data_behavior="overwrite_or_ignore"
#     )

# def main():
#     os.makedirs(OUTPUT_DIR, exist_ok=True)

#     # Load progress
#     progress = load_progress()

#     # Get total size and list of files
#     total_size, all_files = get_total_size_and_files(DATA_DIR)
#     processed_size = progress["processed_size"]
#     processed_files = set(progress["processed_files"])

#     print(f"Total files: {len(all_files)}, Total size: {total_size / (1024 ** 3):.2f} GB")

#     # Initialize progress bar
#     with tqdm(total=total_size, initial=processed_size, unit="B", unit_scale=True, desc="Processing JSONs") as pbar:
#         for file_path in all_files:
#             file_name = Path(file_path).name

#             if file_name in processed_files:
#                 continue  # Skip already processed files

#             try:
#                 # Convert JSON to Pandas DataFrame
#                 df = process_file_to_dataframe(file_path)

#                 # Save to partitioned Parquet
#                 save_to_parquet(df, OUTPUT_DIR, partition_column=PARTITION_COLUMN)

#                 file_size = os.path.getsize(file_path)
#                 processed_size += file_size
#                 processed_files.add(file_name)

#                 # Update progress
#                 progress["processed_size"] = processed_size
#                 progress["processed_files"] = list(processed_files)
#                 save_progress(progress)

#                 pbar.update(file_size)
#             except Exception as e:
#                 error_message = str(e)
#                 log_error(file_path, error_message)
#                 print(f"Error processing {file_path}: {error_message}")

#     # Verify completion
#     completion_percentage = len(processed_files) / len(all_files) * 100
#     print(f"Processing complete! {completion_percentage:.2f}% of files have been converted to Parquet.")
#     if len(processed_files) < len(all_files):
#         print(f"Missing files: {len(all_files) - len(processed_files)}")

# if __name__ == "__main__":
#     main()


In [2]:
# import os
# import json
# from pathlib import Path

# # Directories and progress file
# OUTPUT_DIR = "parquet_data"
# PROGRESS_FILE = "progress_log.json"

# def delete_parquet_files(output_dir):
#     """Delete all Parquet files in the output directory."""
#     if os.path.exists(output_dir):
#         for root, _, files in os.walk(output_dir):
#             for file in files:
#                 if file.endswith(".parquet"):
#                     file_path = os.path.join(root, file)
#                     try:
#                         os.remove(file_path)
#                         print(f"Deleted: {file_path}")
#                     except Exception as e:
#                         print(f"Error deleting {file_path}: {e}")
#         print(f"All Parquet files in '{output_dir}' have been deleted.")
#     else:
#         print(f"Output directory '{output_dir}' does not exist.")

# def reset_progress(progress_file):
#     """Reset the progress log file."""
#     if os.path.exists(progress_file):
#         try:
#             os.remove(progress_file)
#             print(f"Progress log '{progress_file}' has been deleted.")
#         except Exception as e:
#             print(f"Error deleting progress log '{progress_file}': {e}")
#     else:
#         print(f"Progress log '{progress_file}' does not exist.")

# def main():
#     # Delete all Parquet files
#     delete_parquet_files(OUTPUT_DIR)

#     # Reset progress log
#     reset_progress(PROGRESS_FILE)

# if __name__ == "__main__":
#     main()


In [3]:
import dask.dataframe as dd

# Load the consolidated Parquet dataset
df = dd.read_parquet("parquet_database")

# Count total records
total_records = len(df)
print(f"Total records: {total_records}")


Total records: 6322335


In [4]:
import dask.dataframe as dd

# Load the consolidated Parquet dataset
df = dd.read_parquet("parquet_database")

# Count total records
total_records = len(df)
print(f"Total records: {total_records}")


Total records: 6322335


In [5]:
import pyarrow.parquet as pq

# Inspect schema of a single file
schema = pq.read_schema('parquet_database/receivedate=1989-12/d6b1934e8fb147998a644a3f49ddf8b7-0.parquet')
print(schema)

safetyreportversion: string
safetyreportid: string
primarysourcecountry: string
occurcountry: string
transmissiondateformat: string
transmissiondate: string
serious: string
seriousnessother: string
receivedateformat: string
receiptdateformat: string
receiptdate: string
fulfillexpeditecriteria: string
companynumb: string
duplicate: string
reportduplicate.duplicatesource: string
reportduplicate.duplicatenumb: string
primarysource.reportercountry: string
primarysource.qualification: string
sender.sendertype: string
sender.senderorganization: string
receiver.receivertype: string
receiver.receiverorganization: string
patient.reaction: list<element: struct<reactionmeddrapt: string, reactionmeddraversionpt: string, reactionoutcome: string>>
  child 0, element: struct<reactionmeddrapt: string, reactionmeddraversionpt: string, reactionoutcome: string>
      child 0, reactionmeddrapt: string
      child 1, reactionmeddraversionpt: string
      child 2, reactionoutcome: string
patient.drug: list<

In [6]:
import duckdb

# Connect to DuckDB
conn = duckdb.connect()

# Inspect schema of the Parquet dataset
schema = conn.execute("""
    DESCRIBE SELECT * FROM parquet_scan('parquet_database/*/*.parquet', union_by_name=True)
""").fetchdf()

print(schema)


                                    column_name  \
0                                safetyreportid   
1                        transmissiondateformat   
2                              transmissiondate   
3                                       serious   
4                             receivedateformat   
5                             receiptdateformat   
6                                   receiptdate   
7                       fulfillexpeditecriteria   
8                                   companynumb   
9                                      receiver   
10                primarysource.reportercountry   
11                  primarysource.qualification   
12                    sender.senderorganization   
13                             patient.reaction   
14                                 patient.drug   
15                      patient.patientonsetage   
16                  patient.patientonsetageunit   
17                        patient.patientweight   
18                           pa

In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
import os

# Directory for Parquet files
PARQUET_DIR = "parquet_database"

# Define a minimal schema for alignment
base_fields = [
    pa.field("reactionmeddrapt", pa.string()),
    pa.field("reactionmeddraversionpt", pa.string()),
    pa.field("reactionoutcome", pa.string()),
    pa.field("receivedate", pa.string()),
    pa.field("patientsex", pa.string())
]
unified_schema = pa.schema(base_fields)

# Align Parquet files to the unified schema
for root, _, files in os.walk(PARQUET_DIR):
    for file in files:
        if file.endswith(".parquet"):
            file_path = os.path.join(root, file)
            table = pq.read_table(file_path)

            # Add missing columns dynamically
            for field in unified_schema:
                if field.name not in table.schema.names:
                    table = table.append_column(field.name, pa.array([None] * len(table)))

            # Write back with the aligned schema
            pq.write_table(table, file_path)


In [8]:
#above content is same as mongodb experiment

In [9]:
# import duckdb

# # Connect to DuckDB
# conn = duckdb.connect()

# # Query the schema of the Parquet dataset
# result = conn.execute("""
#     SELECT * FROM parquet_scan('parquet_database/*/*.parquet') LIMIT 1
# """).fetchdf()

# print("Schema:")
# print(result.dtypes)


In [10]:
# result = conn.execute("PRAGMA show_tables").fetchall()
# print("Tables:", result)


In [11]:
# # Fetch a small sample of the dataset
# sample = conn.execute("SELECT * FROM parquet_scan('parquet_database/*/*.parquet') LIMIT 5").fetchdf()
# print(sample)


In [12]:
total_records = conn.execute("SELECT COUNT(*) FROM parquet_scan('parquet_database/*/*.parquet')").fetchone()[0]
print(f"Total records: {total_records}")


IOException: IO Error: No files found that match the pattern "parquet_database/*/*.parquet"

In [46]:
result = conn.execute("""
    SELECT receiptdate, COUNT(*) as record_count
    FROM parquet_scan('parquet_database/*/*.parquet')
    GROUP BY receiptdate
    ORDER BY receiptdate
""").fetchdf()

print(result)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     receiptdate  record_count
0       00901102             1
1       10100526             1
2       19080729             1
3       19190215             1
4       19281116             1
...          ...           ...
5526    52001024             1
5527    60040726             1
5528    62000410             1
5529    72001111             1
5530    92000502             1

[5531 rows x 2 columns]


In [72]:
# result = conn.execute("""
#     SELECT *
#     FROM parquet_scan('parquet_database/*/*.parquet')
#     WHERE receiptdate = '20040317'
#     LIMIT 10
# """).fetchdf()

# print(result)


In [74]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Big Data Visualization and Analytics") \
    .getOrCreate()

# Load partitioned Parquet dataset
df = spark.read.parquet("parquet_database")

# Display schema
df.printSchema()

# Show sample rows
df.show(5)


24/12/11 07:04:16 WARN Utils: Your hostname, Abhinavs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.16 instead (on interface en0)
24/12/11 07:04:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 07:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/11 07:04:20 WARN DataSource: [COLUMN_ALREADY_EXISTS] The column `receivedate` already exists. Consider to choose another name or rename the existing column.


root
 |-- safetyreportid: string (nullable = true)
 |-- transmissiondateformat: string (nullable = true)
 |-- transmissiondate: string (nullable = true)
 |-- serious: string (nullable = true)
 |-- receivedateformat: string (nullable = true)
 |-- receiptdateformat: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- fulfillexpeditecriteria: string (nullable = true)
 |-- companynumb: string (nullable = true)
 |-- receiver: double (nullable = true)
 |-- primarysource.reportercountry: string (nullable = true)
 |-- primarysource.qualification: string (nullable = true)
 |-- sender.senderorganization: string (nullable = true)
 |-- patient.reaction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- reactionmeddrapt: string (nullable = true)
 |    |    |-- reactionmeddraversionpt: string (nullable = true)
 |-- patient.drug: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- activesubstance: struct (nul

24/12/11 07:04:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+----------------------+----------------+-------+-----------------+-----------------+-----------+-----------------------+--------------------+--------+-----------------------------+---------------------------+-------------------------+--------------------+--------------------+-----------------------+---------------------------+---------------------+------------------+--------------------------+--------------------------+----------------+----------------+-------------------------------------------+-------------------------------------+-------------+--------------------+----------------------------+-------------------+--------------------+----------+---------+-------------------------------+-----------------------------+-----------------+---------------------+-----------------------------+----------------------------------------+-----------------------+----------------+-----------------------+---------------+-----------+----------+
|safetyreportid|transmissiondateformat|tr

In [75]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Parquet Analysis") \
    .getOrCreate()

# Load Parquet dataset
df = spark.read.parquet("parquet_database")

# Show schema to verify structure
df.printSchema()


24/12/11 07:04:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- safetyreportid: string (nullable = true)
 |-- transmissiondateformat: string (nullable = true)
 |-- transmissiondate: string (nullable = true)
 |-- serious: string (nullable = true)
 |-- receivedateformat: string (nullable = true)
 |-- receiptdateformat: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- fulfillexpeditecriteria: string (nullable = true)
 |-- companynumb: string (nullable = true)
 |-- receiver: double (nullable = true)
 |-- primarysource.reportercountry: string (nullable = true)
 |-- primarysource.qualification: string (nullable = true)
 |-- sender.senderorganization: string (nullable = true)
 |-- patient.reaction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- reactionmeddrapt: string (nullable = true)
 |    |    |-- reactionmeddraversionpt: string (nullable = true)
 |-- patient.drug: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- activesubstance: struct (nul

24/12/11 07:04:24 WARN DataSource: [COLUMN_ALREADY_EXISTS] The column `receivedate` already exists. Consider to choose another name or rename the existing column.


In [76]:
# 1. Count Total Reports Over Time (OLAP)

In [77]:
from pyspark.sql.functions import year, month, col, concat_ws

# Add Year-Month column
reports_over_time = (
    df.withColumn("YearMonth", concat_ws("-", year(col("receivedate")), month(col("receivedate"))))
    .groupBy("YearMonth")
    .count()
    .orderBy("YearMonth")
)

# Convert to Pandas for visualization
reports_over_time_pd = reports_over_time.toPandas()

# Plot using Matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(reports_over_time_pd["YearMonth"], reports_over_time_pd["count"], marker='o')
plt.xlabel("Year-Month")
plt.ylabel("Number of Reports")
plt.title("Number of Reports Over Time (Aggregated by Year-Month)")
plt.xticks(rotation=45, ticks=reports_over_time_pd["YearMonth"][::12])  # Show only yearly ticks
plt.grid()
plt.tight_layout()
plt.show()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x0000000103db911c, pid=84654, tid=69635
#
# JRE version: OpenJDK Runtime Environment Homebrew (11.0.25) (build 11.0.25+0)
# Java VM: OpenJDK 64-Bit Server VM Homebrew (11.0.25+0, mixed mode, tiered, compressed oops, g1 gc, bsd-aarch64)
# Problematic frame:
# V  [libjvm.dylib+0x69511c]  ObjectSynchronizer::inflate(Thread*, oopDesc*, ObjectSynchronizer::InflateCause)+0x18c
#
# No core dump will be written. Core dumps have been disabled. To enable core dumping, try "ulimit -c unlimited" before starting Java again
#
[thread 84231 also had an error]
# An error report file with more information is saved as:
# /Volumes/abhinav/Project/hs_err_pid84654.log
#
# If you would like to submit a bug report, please visit:
#   https://github.com/Homebrew/homebrew-core/issues
#


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# 2. Top 10 Most Common Reactions

In [ ]:
from pyspark.sql.functions import col, explode

# Explode the `patient.reaction` array and count occurrences of `reactionmeddrapt`
reaction_counts = (
    df.select(explode(col("`patient.reaction`")).alias("reaction"))  # Flatten the array
    .groupBy(col("reaction.reactionmeddrapt"))  # Group by `reactionmeddrapt`
    .count()  # Count occurrences
    .orderBy(col("count").desc())  # Order by count descending
    .limit(10)  # Top 10 reactions
)

# Convert to Pandas for visualization
reaction_counts_pd = reaction_counts.toPandas()

# Plot the results using Matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(reaction_counts_pd["reactionmeddrapt"], reaction_counts_pd["count"], color='skyblue')
plt.xlabel("Count")
plt.ylabel("Reaction")
plt.title("Top 10 Most Common Reactions")
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.tight_layout()
plt.show()


In [ ]:
# 3. Top 10 Drugs by Count


In [ ]:
from pyspark.sql.functions import col, explode

# Explode the `patient.drug` array and count occurrences of `medicinalproduct`
drug_counts = (
    df.select(explode(col("`patient.drug`")).alias("drug"))  # Flatten the array
    .groupBy(col("drug.medicinalproduct"))  # Group by `medicinalproduct`
    .count()  # Count occurrences
    .orderBy(col("count").desc())  # Order by count descending
    .limit(10)  # Top 10 drugs
)

# Convert to Pandas for visualization
drug_counts_pd = drug_counts.toPandas()

# Plot the results using Matplotlib
plt.figure(figsize=(10, 6))
plt.barh(drug_counts_pd["medicinalproduct"], drug_counts_pd["count"], color='lightcoral')
plt.xlabel("Count")
plt.ylabel("Drug")
plt.title("Top 10 Most Common Drugs")
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.tight_layout()
plt.show()


In [ ]:
# 4. Distribution of Seriousness


In [ ]:
import matplotlib.pyplot as plt

# Plot using the `serious` column directly
plt.figure(figsize=(8, 6))
plt.bar(seriousness_pd["serious"], seriousness_pd["count"], color='steelblue')

# Customize x-axis labels to align with bar positions
plt.xticks(ticks=seriousness_pd["serious"], labels=["Low", "High"])  # Map 1 -> Low, 2 -> High

# Add axis labels and title
plt.xlabel("Seriousness Level")
plt.ylabel("Count")
plt.title("Distribution of Seriousness")

# Add grid for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Ensure layout is tight
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
# 5. Reaction Trends Over Time


In [ ]:
from pyspark.sql.functions import col, explode, year, month, concat_ws
import matplotlib.pyplot as plt

# Aggregate by Year-Month
df = df.withColumn("YearMonth", concat_ws("-", year(col("receivedate")), month(col("receivedate"))))

# Explode `patient.reaction` and group by YearMonth
reaction_trends = (
    df.select(col("YearMonth"), explode(col("`patient.reaction`")).alias("reaction"))
    .groupBy("YearMonth", "reaction.reactionmeddrapt")  # Group by YearMonth and reaction
    .count()
    .orderBy("YearMonth")
)

# Convert to Pandas for further analysis
reaction_trends_pd = reaction_trends.toPandas()

# Focus on top N reactions globally
top_reactions = (
    reaction_trends_pd.groupby("reactionmeddrapt")["count"].sum().nlargest(5).index
)
reaction_trends_filtered = reaction_trends_pd[
    reaction_trends_pd["reactionmeddrapt"].isin(top_reactions)
]

# Pivot for visualization
reaction_pivot = reaction_trends_filtered.pivot(
    index="YearMonth", columns="reactionmeddrapt", values="count"
).fillna(0)

# Plot top reactions
plt.figure(figsize=(14, 7))
for reaction in reaction_pivot.columns:
    plt.plot(reaction_pivot.index, reaction_pivot[reaction], label=reaction)

# Customize the plot
plt.xlabel("Year-Month")
plt.ylabel("Count")
plt.title("Reaction Trends Over Time (Aggregated by Year-Month)")
plt.legend(title="Reactions", loc='upper left', bbox_to_anchor=(1, 1))
plt.grid()
plt.xticks(rotation=45)

# Ensure layout is tight for better display
plt.tight_layout()
plt.show()


In [ ]:
# 4. Patient Sex Distribution


In [ ]:
print(patient_sex_counts_pd.columns)


In [ ]:
# Group by patientsex and count
patient_sex_counts = df.groupBy(col("`patient.patientsex`")).count()

# Convert to Pandas for visualization
patient_sex_counts_pd = patient_sex_counts.toPandas()

# Fix column names in the Pandas DataFrame
labels = patient_sex_counts_pd["patient.patientsex"]
sizes = patient_sex_counts_pd["count"]

# Plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Patient Sex")
plt.show()



In [ ]:
# 5. Average Age of Patients by Reaction


In [ ]:
from pyspark.sql.functions import col, explode, avg, when

# Normalize patient age based on patientonsetageunit
df = df.withColumn(
    "normalized_age",
    when(col("`patient.patientonsetageunit`") == "801", col("`patient.patientonsetage`").cast("double"))  # Years
    .when(col("`patient.patientonsetageunit`") == "802", col("`patient.patientonsetage`").cast("double") / 12)  # Months -> years
    .when(col("`patient.patientonsetageunit`") == "803", col("`patient.patientonsetage`").cast("double") / 365)  # Days -> years
    .otherwise(None)
)

# Calculate average age for each reaction type
avg_age_by_reaction = (
    df.select(
        explode(col("`patient.reaction`")).alias("reaction"),  # Explode reaction array
        col("normalized_age").alias("patientonsetage")
    )
    .groupBy("reaction.reactionmeddrapt")
    .agg(avg("patientonsetage").alias("avg_age"))
    .orderBy("avg_age", ascending=False)
    .limit(10)
)

# Convert to Pandas for visualization
avg_age_by_reaction_pd = avg_age_by_reaction.toPandas()

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(avg_age_by_reaction_pd["reactionmeddrapt"], avg_age_by_reaction_pd["avg_age"])
plt.xlabel("Average Age")
plt.ylabel("Reaction")
plt.title("Top 10 Reactions by Average Patient Age")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
df.printSchema()


In [ ]:
# 6. Seriousness Impact on Deaths


In [ ]:
# Group by seriousness and seriousnessdeath
seriousness_impact = (
    df.groupBy("serious", "seriousnessdeath")
    .count()
    .orderBy("serious", "seriousnessdeath")
)

# Convert to Pandas for visualization
seriousness_impact_pd = seriousness_impact.toPandas()

# Pivot for heatmap visualization
import seaborn as sns

seriousness_pivot = seriousness_impact_pd.pivot(index="serious", columns="seriousnessdeath", values="count").fillna(0)

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(seriousness_pivot, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title("Impact of Seriousness Levels on Deaths")
plt.xlabel("Seriousness Death")
plt.ylabel("Seriousness Level")
plt.show()


In [ ]:
df.printSchema()


In [ ]:
# %pip install duckdb


In [ ]:
# import duckdb

# # Check DuckDB version
# print(duckdb.__version__)

In [ ]:
import sys
sys.executable #base env

In [ ]:
!which python
!which java
!echo $JAVA_HOME

In [ ]:
import pyspark
print(f"Spark version: {pyspark.__version__}")
